In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import scipy.integrate as integrate
import ipywidgets as widgets
import scipy.special as spe

from matplotlib import cm, colors
from mpl_toolkits.mplot3d import Axes3D

In [2]:
def psi_R(r,n=1,l=0):

    coeff = np.sqrt((2.0/n)**3 * spe.factorial(n-l-1) /(2.0*n*spe.factorial(n+l)))
    laguerre = spe.assoc_laguerre(2.0*r/n,n-l-1,2*l+1)
    
    return coeff * np.exp(-r/n) * (2.0*r/n)**l * laguerre

In [3]:
nmax=6

@widgets.interact(n = np.arange(1,nmax,1), l = np.arange(0,nmax-1,1))

def plot_radial(n=3,l=0):
    
    r = np.linspace(0,250,10000)
    psi2 = psi_R(r,n,l)**2 * (r**2)
    plt.plot(r, psi2, lw=2, color='red')
    

    ''' Styling the plot'''
    
    plt.xlabel('$r [a_0]$')
    plt.ylabel('$R_{nl}(r)$')
    rmax = n**2*(1+0.5*(1-l*(l+1)/n**2))
    
    plt.xlim([0, 2*rmax])

interactive(children=(Dropdown(description='n', index=2, options=(1, 2, 3, 4, 5), value=3), Dropdown(descripti…

In [4]:
def psi_ang(phi,theta,l=0,m=0):
    
    sphHarm = spe.sph_harm(m,l,phi,theta)
    
    return sphHarm.real

In [5]:
nmax=6
lmax = nmax-1

@widgets.interact(l = np.arange(0,nmax-1,1), m=np.arange(-lmax+1,lmax,1))

def plot_angular(l=2,m=0):
    
    phi, theta = np.linspace(0, np.pi, 100), np.linspace(0, 2*np.pi, 100)
    phi, theta = np.meshgrid(phi, theta)
    Ylm = psi_ang(theta,phi,l,m)

    x = np.sin(phi) * np.cos(theta) * abs(Ylm)
    y = np.sin(phi) * np.sin(theta) * abs(Ylm)
    z = np.cos(phi) * abs(Ylm)

    fig = plt.figure(figsize=(9,9))
    ax = fig.add_subplot(111, projection='3d')


    fcolors = (Ylm - Ylm.min())/(Ylm.max() - Ylm.min())
    ax.plot_surface(x, y, z, facecolors=cm.seismic(fcolors), alpha=0.3)

    cset = ax.contour(x, y, z,20, zdir='z',offset = -1, cmap='summer')
    cset = ax.contour(x, y, z,20, zdir='y',offset =  1, cmap='winter' )
    cset = ax.contour(x, y, z,20, zdir='x',offset = -1, cmap='autumn')

    ax.set_xlim(-1, 1)
    ax.set_ylim(-1, 1)
    ax.set_zlim(-1, 1)


interactive(children=(Dropdown(description='l', index=2, options=(0, 1, 2, 3, 4), value=2), Dropdown(descripti…

In [6]:
def HFunc(r,theta,phi,n,l,m):

    return psi_R(r,n,l) * psi_ang(phi,theta,l,m)


In [7]:
nmax = 10
lmax = nmax-1

@widgets.interact(n=np.arange(1,nmax,1), l = np.arange(0,nmax-1,1), m=np.arange(-lmax,lmax+1,1))

def psi_xz_plot(n=3,l=2,m=0):

    plt.figure(figsize=(10,5))
    limit = 4*(n+l) 
    
    x_1d = np.linspace(-limit,limit,500)
    y_1d = np.linspace(-limit,limit,500)
    z_1d = np.linspace(-limit,limit,500)
    
    x1,y1 = np.meshgrid(x_1d,y_1d)
    z1 = 0
    
    x2,z2 = np.meshgrid(x_1d,z_1d)
    y2 = 0
    
    y3,z3 = np.meshgrid(y_1d,z_1d)
    x3 = 0
    
    
    r1     = np.sqrt(x1**2 + y1**2 + z1**2)
    theta1 = np.arctan2(np.sqrt(x1**2+y1**2), z1 )
    phi1   = np.arctan2(y1, x1)
    
    r2     = np.sqrt(x2**2 + y2**2 + z2**2)
    theta2 = np.arctan2(np.sqrt(x2**2+y2**2), z2 )
    phi2   = np.arctan2(y2, x2)

    r3     = np.sqrt(x3**2 + y3**2 + z3**2)
    theta3 = np.arctan2(np.sqrt(x3**2+y3**2), z3 )
    phi3   = np.arctan2(y3, x3)


    psi_nlm1 = HFunc(r1,theta1,phi1,n,l,m)
    psi_nlm2 = HFunc(r2,theta2,phi2,n,l,m)
    psi_nlm3 = HFunc(r3,theta3,phi3,n,l,m)
    
    fig, axs = plt.subplots(3,figsize=(6,19))
   
    axs[2].contourf(x1, y1,  psi_nlm1, 25, cmap='seismic', alpha=0.7)  
    axs[1].contourf(x2, z2,  psi_nlm2, 25, cmap='seismic', alpha=0.7)  # Classic orbitals
    axs[0].contourf(y3, z3,  psi_nlm3, 25, cmap='seismic', alpha=0.7)  # Classic orbitals
    

interactive(children=(Dropdown(description='n', index=2, options=(1, 2, 3, 4, 5, 6, 7, 8, 9), value=3), Dropdo…